# Lesson I1: Structs & Methods

**Duration**: 90-105 minutes  
**Stage**: Intermediate (Building Skills)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Define and instantiate structs to model complex data
2. Implement methods and associated functions for structs
3. Use multiple `impl` blocks to organize functionality
4. Apply struct update syntax and field init shorthand
5. Design data structures that follow Rust ownership principles

---

## 📋 Prerequisite Review

**Quick Check**: From the beginner stage:

1. What are the three ownership rules?
2. How do you create a mutable reference?
3. What's the difference between `String` and `&str`?
4. How do you handle errors using `Result<T, E>`?

**Answers**: 1) Each value has one owner, only one owner at a time, value dropped when owner goes out of scope, 2) `&mut variable`, 3) `String` is owned, `&str` is borrowed, 4) Use `match` or methods like `unwrap()`, `expect()`, `?`

---

## 🧠 Key Concepts

### What are Structs?

**Structs** are custom data types that group related data together:
- **Named fields**: Each piece of data has a descriptive name
- **Type safety**: Each field has a specific type
- **Ownership**: Structs own their data by default
- **Methods**: Functions associated with the struct type

### Three Types of Structs

1. **Named Field Structs**: Most common, fields have names
2. **Tuple Structs**: Fields accessed by position
3. **Unit Structs**: No fields, useful for traits

### Methods vs Associated Functions

- **Methods**: Take `&self`, `&mut self`, or `self` as first parameter
- **Associated Functions**: Don't take `self`, often used as constructors

---

## 🔬 Live Code Exploration

### Basic Struct Definition and Usage

In [ ]:
// Define a struct to represent a user
#[derive(Debug)]  // Allows us to print the struct
struct User {
    username: String,
    email: String,
    age: u32,
    active: bool,
}

fn basic_struct_usage() {
    // Create an instance
    let user1 = User {
        username: String::from("alice123"),
        email: String::from("alice@example.com"),
        age: 25,
        active: true,
    };
    
    println!("User: {:?}", user1);
    println!("Username: {}", user1.username);
    println!("Email: {}", user1.email);
    
    // Mutable struct
    let mut user2 = User {
        username: String::from("bob456"),
        email: String::from("bob@example.com"),
        age: 30,
        active: false,
    };
    
    // Modify a field
    user2.active = true;
    user2.age += 1;
    
    println!("Updated user: {:?}", user2);
}

basic_struct_usage();

### Struct Methods and Associated Functions

In [ ]:
#[derive(Debug)]
struct Rectangle {
    width: f64,
    height: f64,
}

impl Rectangle {
    // Associated function (constructor)
    fn new(width: f64, height: f64) -> Self {
        Rectangle { width, height }  // Field init shorthand
    }
    
    // Associated function for creating a square
    fn square(size: f64) -> Self {
        Rectangle {
            width: size,
            height: size,
        }
    }
    
    // Method that borrows self immutably
    fn area(&self) -> f64 {
        self.width * self.height
    }
    
    // Method that borrows self immutably
    fn perimeter(&self) -> f64 {
        2.0 * (self.width + self.height)
    }
    
    // Method that borrows self immutably
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width >= other.width && self.height >= other.height
    }
    
    // Method that borrows self mutably
    fn scale(&mut self, factor: f64) {
        self.width *= factor;
        self.height *= factor;
    }
    
    // Method that takes ownership of self
    fn into_square(self) -> Rectangle {
        let size = self.width.max(self.height);
        Rectangle::square(size)
    }
}

fn struct_methods_demo() {
    // Using associated functions
    let rect1 = Rectangle::new(10.0, 20.0);
    let square1 = Rectangle::square(15.0);
    
    println!("Rectangle: {:?}", rect1);
    println!("Square: {:?}", square1);
    
    // Using methods
    println!("Rectangle area: {:.2}", rect1.area());
    println!("Rectangle perimeter: {:.2}", rect1.perimeter());
    println!("Square area: {:.2}", square1.area());
    
    // Method with multiple parameters
    println!("Can rectangle hold square? {}", rect1.can_hold(&square1));
    println!("Can square hold rectangle? {}", square1.can_hold(&rect1));
    
    // Mutable method
    let mut rect2 = Rectangle::new(5.0, 8.0);
    println!("Before scaling: {:?}", rect2);
    rect2.scale(2.0);
    println!("After scaling: {:?}", rect2);
    
    // Method that takes ownership
    let rect3 = Rectangle::new(3.0, 7.0);
    let new_square = rect3.into_square();  // rect3 is moved here
    println!("Converted to square: {:?}", new_square);
    // rect3 is no longer accessible
}

struct_methods_demo();

### Multiple Implementation Blocks

In [ ]:
#[derive(Debug, Clone)]
struct BankAccount {
    account_number: String,
    holder_name: String,
    balance: f64,
    is_active: bool,
}

// First impl block: Constructors and basic operations
impl BankAccount {
    fn new(account_number: String, holder_name: String) -> Self {
        BankAccount {
            account_number,
            holder_name,
            balance: 0.0,
            is_active: true,
        }
    }
    
    fn balance(&self) -> f64 {
        self.balance
    }
    
    fn is_active(&self) -> bool {
        self.is_active
    }
}

// Second impl block: Transaction operations
impl BankAccount {
    fn deposit(&mut self, amount: f64) -> Result<(), String> {
        if !self.is_active {
            return Err("Account is not active".to_string());
        }
        
        if amount <= 0.0 {
            return Err("Deposit amount must be positive".to_string());
        }
        
        self.balance += amount;
        Ok(())
    }
    
    fn withdraw(&mut self, amount: f64) -> Result<(), String> {
        if !self.is_active {
            return Err("Account is not active".to_string());
        }
        
        if amount <= 0.0 {
            return Err("Withdrawal amount must be positive".to_string());
        }
        
        if amount > self.balance {
            return Err("Insufficient funds".to_string());
        }
        
        self.balance -= amount;
        Ok(())
    }
}

// Third impl block: Account management
impl BankAccount {
    fn close_account(&mut self) {
        self.is_active = false;
    }
    
    fn reopen_account(&mut self) {
        self.is_active = true;
    }
    
    fn transfer_to(&mut self, other: &mut BankAccount, amount: f64) -> Result<(), String> {
        self.withdraw(amount)?;  // ? operator propagates errors
        
        match other.deposit(amount) {
            Ok(()) => Ok(()),
            Err(e) => {
                // Rollback the withdrawal
                self.balance += amount;
                Err(format!("Transfer failed: {}", e))
            }
        }
    }
}

fn multiple_impl_blocks_demo() {
    let mut account1 = BankAccount::new(
        "ACC001".to_string(),
        "Alice Johnson".to_string()
    );
    
    let mut account2 = BankAccount::new(
        "ACC002".to_string(),
        "Bob Smith".to_string()
    );
    
    println!("Initial accounts:");
    println!("Account 1: {:?}", account1);
    println!("Account 2: {:?}", account2);
    
    // Deposit money
    match account1.deposit(1000.0) {
        Ok(()) => println!("Deposited $1000 to account 1"),
        Err(e) => println!("Deposit failed: {}", e),
    }
    
    match account2.deposit(500.0) {
        Ok(()) => println!("Deposited $500 to account 2"),
        Err(e) => println!("Deposit failed: {}", e),
    }
    
    println!("\nAfter deposits:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
    
    // Transfer money
    match account1.transfer_to(&mut account2, 200.0) {
        Ok(()) => println!("\nTransferred $200 from account 1 to account 2"),
        Err(e) => println!("\nTransfer failed: {}", e),
    }
    
    println!("\nAfter transfer:");
    println!("Account 1 balance: ${:.2}", account1.balance());
    println!("Account 2 balance: ${:.2}", account2.balance());
    
    // Try to withdraw too much
    match account1.withdraw(1000.0) {
        Ok(()) => println!("Withdrawal successful"),
        Err(e) => println!("\nWithdrawal failed: {}", e),
    }
}

multiple_impl_blocks_demo();

### Struct Update Syntax and Field Init Shorthand

In [ ]:
#[derive(Debug, Clone)]
struct Config {
    debug: bool,
    max_connections: u32,
    timeout_seconds: u64,
    server_name: String,
    port: u16,
}

impl Config {
    fn default() -> Self {
        Config {
            debug: false,
            max_connections: 100,
            timeout_seconds: 30,
            server_name: String::from("localhost"),
            port: 8080,
        }
    }
    
    // Constructor using field init shorthand
    fn new(server_name: String, port: u16, debug: bool) -> Self {
        Config {
            server_name,  // Shorthand for server_name: server_name
            port,         // Shorthand for port: port
            debug,        // Shorthand for debug: debug
            ..Config::default()  // Use default values for remaining fields
        }
    }
    
    fn production_config(&self) -> Self {
        Config {
            debug: false,
            max_connections: 1000,
            timeout_seconds: 60,
            ..self.clone()  // Copy other fields from self
        }
    }
}

fn struct_update_syntax_demo() {
    // Default configuration
    let default_config = Config::default();
    println!("Default config: {:?}", default_config);
    
    // Custom configuration using field init shorthand
    let dev_config = Config::new(
        String::from("dev.example.com"),
        3000,
        true
    );
    println!("\nDev config: {:?}", dev_config);
    
    // Production configuration using struct update syntax
    let prod_config = dev_config.production_config();
    println!("\nProduction config: {:?}", prod_config);
    
    // Create a variant with just one field changed
    let test_config = Config {
        port: 4000,
        ..default_config  // Use all other fields from default_config
    };
    println!("\nTest config: {:?}", test_config);
    
    // Note: default_config was moved in the previous line
    // This would cause an error: println!("{:?}", default_config);
}

struct_update_syntax_demo();

---

## 🎯 Guided Practice

### Exercise 1: Student Management System

Create a struct to represent students and implement methods for grade management.

In [ ]:
// TODO: Complete the Student struct and its implementation

#[derive(Debug, Clone)]
struct Student {
    id: u32,
    name: String,
    email: String,
    grades: Vec<f64>,
}

impl Student {
    // TODO: Implement constructor
    fn new(id: u32, name: String, email: String) -> Self {
        // Create a new student with empty grades
        Student {
            id,
            name,
            email,
            grades: Vec::new(),
        }
    }
    
    // TODO: Add a grade
    fn add_grade(&mut self, grade: f64) -> Result<(), String> {
        // Validate grade is between 0.0 and 100.0
        if grade < 0.0 || grade > 100.0 {
            return Err("Grade must be between 0.0 and 100.0".to_string());
        }
        self.grades.push(grade);
        Ok(())
    }
    
    // TODO: Calculate average grade
    fn average_grade(&self) -> Option<f64> {
        if self.grades.is_empty() {
            None
        } else {
            let sum: f64 = self.grades.iter().sum();
            Some(sum / self.grades.len() as f64)
        }
    }
    
    // TODO: Get letter grade based on average
    fn letter_grade(&self) -> Option<char> {
        self.average_grade().map(|avg| {
            match avg {
                90.0..=100.0 => 'A',
                80.0..=89.9 => 'B',
                70.0..=79.9 => 'C',
                60.0..=69.9 => 'D',
                _ => 'F',
            }
        })
    }
    
    // TODO: Check if student is passing (average >= 60.0)
    fn is_passing(&self) -> bool {
        self.average_grade().map_or(false, |avg| avg >= 60.0)
    }
}

fn student_management_demo() {
    let mut student = Student::new(
        12345,
        "Alice Johnson".to_string(),
        "alice@university.edu".to_string()
    );
    
    println!("New student: {:?}", student);
    
    // Add some grades
    let grades = [85.5, 92.0, 78.5, 88.0, 95.5];
    for grade in &grades {
        match student.add_grade(*grade) {
            Ok(()) => println!("Added grade: {}", grade),
            Err(e) => println!("Failed to add grade: {}", e),
        }
    }
    
    // Display student information
    println!("\nStudent grades: {:?}", student.grades);
    
    if let Some(avg) = student.average_grade() {
        println!("Average grade: {:.2}", avg);
    }
    
    if let Some(letter) = student.letter_grade() {
        println!("Letter grade: {}", letter);
    }
    
    println!("Is passing: {}", student.is_passing());
    
    // Try to add an invalid grade
    match student.add_grade(105.0) {
        Ok(()) => println!("Added invalid grade"),
        Err(e) => println!("\nError adding invalid grade: {}", e),
    }
}

student_management_demo();

---

## 🚀 Independent Practice

### Challenge: Library Management System

Create a more complex system with multiple related structs.

In [ ]:
// TODO: Implement a library management system

#[derive(Debug, Clone)]
struct Book {
    isbn: String,
    title: String,
    author: String,
    year: u32,
    is_available: bool,
}

#[derive(Debug)]
struct Library {
    name: String,
    books: Vec<Book>,
}

// TODO: Implement methods for Book
impl Book {
    fn new(isbn: String, title: String, author: String, year: u32) -> Self {
        // TODO: Create a new book (available by default)
        Book {
            isbn,
            title,
            author,
            year,
            is_available: true,
        }
    }
    
    fn borrow_book(&mut self) -> Result<(), String> {
        // TODO: Mark book as borrowed if available
        if !self.is_available {
            Err("Book is already borrowed".to_string())
        } else {
            self.is_available = false;
            Ok(())
        }
    }
    
    fn return_book(&mut self) -> Result<(), String> {
        // TODO: Mark book as available if borrowed
        if self.is_available {
            Err("Book is not currently borrowed".to_string())
        } else {
            self.is_available = true;
            Ok(())
        }
    }
}

// TODO: Implement methods for Library
impl Library {
    fn new(name: String) -> Self {
        // TODO: Create a new library with empty book collection
        Library {
            name,
            books: Vec::new(),
        }
    }
    
    fn add_book(&mut self, book: Book) {
        // TODO: Add a book to the library
        self.books.push(book);
    }
    
    fn find_book_by_isbn(&mut self, isbn: &str) -> Option<&mut Book> {
        // TODO: Find a book by ISBN
        self.books.iter_mut().find(|book| book.isbn == isbn)
    }
    
    fn available_books(&self) -> Vec<&Book> {
        // TODO: Return a vector of references to available books
        self.books.iter().filter(|book| book.is_available).collect()
    }
    
    fn books_by_author(&self, author: &str) -> Vec<&Book> {
        // TODO: Return books by a specific author
        self.books.iter().filter(|book| book.author == author).collect()
    }
}

fn library_management_demo() {
    let mut library = Library::new("Central Library".to_string());
    
    // Add some books
    library.add_book(Book::new(
        "978-0134685991".to_string(),
        "Effective Modern C++".to_string(),
        "Scott Meyers".to_string(),
        2014
    ));
    
    library.add_book(Book::new(
        "978-1718500440".to_string(),
        "The Rust Programming Language".to_string(),
        "Steve Klabnik".to_string(),
        2019
    ));
    
    library.add_book(Book::new(
        "978-0134494166".to_string(),
        "Clean Code".to_string(),
        "Robert C. Martin".to_string(),
        2008
    ));
    
    println!("Library: {}", library.name);
    println!("Total books: {}", library.books.len());
    
    // Show available books
    let available = library.available_books();
    println!("\nAvailable books: {}", available.len());
    for book in &available {
        println!("  - {} by {}", book.title, book.author);
    }
    
    // Borrow a book
    if let Some(book) = library.find_book_by_isbn("978-1718500440") {
        match book.borrow_book() {
            Ok(()) => println!("\nBorrowed: {}", book.title),
            Err(e) => println!("\nFailed to borrow: {}", e),
        }
    }
    
    // Show available books after borrowing
    let available_after = library.available_books();
    println!("\nAvailable books after borrowing: {}", available_after.len());
    for book in &available_after {
        println!("  - {} by {}", book.title, book.author);
    }
}

library_management_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What's the difference between a method and an associated function?
2. How do you create a constructor for a struct?
3. What does the `&self` parameter mean in a method?
4. When would you use `&mut self` vs `self` as a parameter?
5. What is struct update syntax and when is it useful?
6. What is field init shorthand?
7. Can you have multiple `impl` blocks for the same struct?
8. How do you make a struct printable with `println!`?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do structs help you organize and model real-world data?**
2. **What are the advantages of having methods associated with data?**
3. **How does Rust's ownership system affect struct design?**
4. **When would you choose to take ownership vs borrow in method parameters?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Enums & Advanced Pattern Matching

In our next lesson, you'll learn about:
- Custom enums with data-carrying variants
- Advanced pattern matching with match guards
- Destructuring complex data structures
- The power of Rust's type system for modeling states

### How This Connects
Structs are fundamental to understanding:
- How enums can contain struct data
- Generic structs and their type parameters
- Trait implementations for custom types
- Error handling with custom error structs

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Define structs with appropriate field types?
- [ ] Implement methods and associated functions?
- [ ] Use multiple impl blocks to organize functionality?
- [ ] Apply struct update syntax and field init shorthand?
- [ ] Choose appropriate method parameter types (&self, &mut self, self)?
- [ ] Handle ownership correctly in struct methods?
- [ ] Design structs that model real-world entities?

If you checked all boxes, excellent! You're ready to tackle more advanced type system features.

---

**🎉 Great Progress!** You now understand how to create custom data types and organize functionality - essential skills for building larger Rust applications!